# Classificação de noticias com Word2Vec e Naive Bayes (usando CSV)

Este notebook realiza a classificação de noticias (feliz ou triste) com base em textos usando técnicas de **Processamento de Linguagem Natural (PLN)** com **Word2Vec** e **Naive Bayes**. As etapas incluem:
- Leitura da base de dados (CSV)
- Limpeza e tokenização dos textos com remoção de stopwords (português)
- Vetorização com Word2Vec (média dos vetores de palavras)
- Treinamento e avaliação com `GaussianNB`
- Predição com frases novas (`teste.txt`)

In [ ]:
# Instalação das bibliotecas necessárias
!pip install pandas scikit-learn
!pip install --upgrade numpy==1.25.2
!pip install --upgrade gensim

In [ ]:
import pandas as pd  # Manipulação de dados com DataFrames (leitura, escrita e organização de tabelas)
import numpy as np  # Biblioteca para operações numéricas e manipulação de arrays
import re  # Expressões regulares para limpeza e pré-processamento de texto
from gensim.models import Word2Vec  # Geração de vetores de palavras a partir do corpo do texto
from sklearn.naive_bayes import GaussianNB  # Classificador Naive Bayes Gaussiano (assume distribuição normal)
from sklearn.model_selection import train_test_split  # Separar os dados em conjuntos de treino e teste
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # Métricas de avaliação do modelo
from imblearn.over_sampling import SMOTE  # Técnica para balancear classes
from sklearn.preprocessing import LabelEncoder  # Codificação de rótulos (strings para inteiros)
from collections import Counter  # Contagem de frequência de elementos, útil para verificar o balanceamento

## Leitura da base CSV

In [ ]:
df = pd.read_csv("noticias_cnj_80.csv")  # Carrega o arquivo CSV com frases (noticia) e classificador (veracidade) em um DataFrame
df.columns = ['noticia', 'veracidade']  # Renomeia as colunas para facilitar a referência no código
df.head(10)  # Exibe as primeiras 10 linhas do DataFrame para inspeção inicial

,noticia,veracidade
0,É falso que Lula recusou vacina japonesa contr...,Falsa
1,OCDE não disse que STF é cúmplice de corrupção,Falsa
2,É falso que app Celular Seguro permite ao gove...,Falsa
3,É falso que Barroso receba mais de R$ 90 mil d...,Falsa
4,É #FAKE que mulher em vídeo ao lado de Dino se...,Falsa
5,Justiça de SP não proibiu leitura da Bíblia em...,Falsa
6,É #FAKE que reforma tributária acabe com direi...,Falsa
7,5 fake news sobre a vacina que previne o HPV,Falsa
8,Artigo 142 não prevê intervenção militar nem f...,Falsa
9,É falso que aplicativo do TSE não permita denú...,Falsa


## Limpeza de texto e tokenização com stopwords

In [ ]:
import re

# Carregar stopwords
with open("stopwords_pt.txt", "r", encoding="ISO-8859-1") as f:
    stopwords = {
        linha.strip() for linha in f
        if linha.strip()
    }

# Padrão que remove TUDO que não for letra (A–Z, à–ÿ) nem espaço
pattern = re.compile(r"[^A-Za-zÀ-ÖØ-öø-ÿ\s]")

def limpar_texto(texto):
    # Se não for string (por ex. NaN), retorna lista vazia
    if not isinstance(texto, str):
        return []
    # Limpeza
    texto = texto.lower()
    texto = pattern.sub(" ", texto)   # substitui por espaço para preservar separação
    # Tokenização e remoção de stopwords
    tokens = [
        palavra for palavra in texto.split()
        if palavra and palavra not in stopwords
    ]
    return tokens

# (Opcional) preencha NaNs antes de aplicar:
# df['noticia'] = df['noticia'].fillna('')

df['tokens'] = df['noticia'].apply(limpar_texto)
df.head(10)


,noticia,veracidade,tokens
0,É falso que Lula recusou vacina japonesa contr...,Falsa,"[é, falso, lula, recusou, vacina, japonesa, de..."
1,OCDE não disse que STF é cúmplice de corrupção,Falsa,"[ocde, disse, stf, é, cúmplice, corrupção]"
2,É falso que app Celular Seguro permite ao gove...,Falsa,"[é, falso, app, celular, seguro, permite, gove..."
3,É falso que Barroso receba mais de R$ 90 mil d...,Falsa,"[é, falso, barroso, receba, mil, vale, aliment..."
4,É #FAKE que mulher em vídeo ao lado de Dino se...,Falsa,"[é, fake, mulher, vídeo, lado, dino, seja, esp..."
5,Justiça de SP não proibiu leitura da Bíblia em...,Falsa,"[justiça, sp, proibiu, leitura, bíblia, sessõe..."
6,É #FAKE que reforma tributária acabe com direi...,Falsa,"[é, fake, reforma, tributária, acabe, direito,..."
7,5 fake news sobre a vacina que previne o HPV,Falsa,"[fake, news, vacina, previne, hpv]"
8,Artigo 142 não prevê intervenção militar nem f...,Falsa,"[artigo, prevê, intervenção, militar, federal]"
9,É falso que aplicativo do TSE não permita denú...,Falsa,"[é, falso, aplicativo, tse, permita, denúncias..."


## Vetorização com Word2Vec (média dos vetores)

In [ ]:
# Gerar sentenças e treinar modelo Word2Vec
sentencas = df['tokens'].tolist()  # Converte a coluna de tokens em uma lista de listas (sentenças)
modelo_w2v = Word2Vec(sentencas, vector_size=100, window=5, min_count=1, workers=4)  # Treina o modelo Word2Vec com vetores de 100 dimensões

# Função para calcular vetor médio da frase
def vetor_frase(tokens):  # Recebe os tokens de uma frase
    vetores = [modelo_w2v.wv[p] for p in tokens if p in modelo_w2v.wv]  # Recupera o vetor de cada palavra presente no vocabulário
    return sum(vetores) / len(vetores) if vetores else [0]*100  # Retorna a média dos vetores ou vetor nulo se estiver vazio

# Aplicar a vetorização
X = df['tokens'].apply(vetor_frase).tolist()  # Aplica a função em cada frase e gera a matriz de vetores
y = df['veracidade']  # Define a variável alvo com os rótulos de veracidade

# Visualizar alguns vetores gerados
print("Exemplo de vetor Word2Vec da primeira frase:")
print(np.round(X[0], 3))  # Mostra o vetor da primeira frase com 3 casas decimais

print("\nFormato da matriz X (frases x dimensões):", np.array(X).shape)  # Mostra o formato da matriz vetorizada

# Visualizar o vetor da palavra "veracidade" (se existir no vocabulário)
if "veracidade" in modelo_w2v.wv:  # Verifica se a palavra "veracidade" está no vocabulário treinado
    print("\nVetor da palavra 'veracidade':")
    print(np.round(modelo_w2v.wv["veracidade"], 3))  # Mostra o vetor da palavra com 3 casas decimais

Exemplo de vetor Word2Vec da primeira frase:
[ 0.002  0.001 -0.001  0.002  0.004  0.001 -0.     0.003 -0.003  0.001
 -0.001  0.001  0.    -0.001  0.001  0.002  0.003  0.002  0.001 -0.002
 -0.002  0.002 -0.003 -0.004  0.    -0.003  0.001  0.    -0.001  0.003
  0.    -0.    -0.001 -0.004  0.     0.002 -0.001 -0.001  0.001 -0.001
 -0.001 -0.001  0.001  0.003  0.     0.003 -0.002 -0.    -0.001  0.002
  0.     0.    -0.002  0.001 -0.002  0.002  0.     0.002  0.     0.
  0.001  0.    -0.    -0.    -0.002  0.001  0.003  0.001 -0.    -0.001
  0.001  0.003  0.004  0.001  0.001 -0.002  0.002  0.001 -0.001  0.
 -0.001  0.004  0.001 -0.001 -0.    -0.002 -0.001 -0.001  0.002 -0.001
 -0.001  0.002 -0.002  0.     0.002  0.001  0.001 -0.002  0.002  0.002]

Formato da matriz X (frases x dimensões): (80, 100)


## Treinamento e avaliação do modelo Naive Bayes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # Divide os dados em 70% treino e 30% teste, com semente fixa para reprodutibilidade
model = GaussianNB()  # Cria o modelo Naive Bayes Gaussiano, ideal para dados contínuos como vetores Word2Vec
model.fit(X_train, y_train)  # Treina o modelo com os dados de treino
y_pred = model.predict(X_test)  # Realiza a predição nos dados de teste com o modelo treinado

print("Acurácia:", accuracy_score(y_test, y_pred))  # Exibe a acurácia do modelo (percentual de acertos)
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))  # Exibe a matriz de confusão (comparação entre classes reais e previstas)
print("Relatório:\n", classification_report(y_test, y_pred))  # Gera o relatório com precision, recall e f1-score para cada classe

Acurácia: 0.9583333333333334
Matriz de Confusão:
 [[17  0]
 [ 1  6]]
Relatório:
               precision    recall  f1-score   support

       Falsa       0.94      1.00      0.97        17
  Verdadeira       1.00      0.86      0.92         7

    accuracy                           0.96        24
   macro avg       0.97      0.93      0.95        24
weighted avg       0.96      0.96      0.96        24



## Predição com frases novas (teste.txt)

In [ ]:
with open("teste.txt", "r", encoding="utf-8") as f:  # Abre o arquivo com frases de teste (uma por linha)
    novas_frases = [linha.split(',')[0].strip("' ") for linha in f.readlines()]  # Lê cada linha e remove aspas ou espaços extras

def vetorizacao_nova(frase):  # Função para limpar e vetorizar uma frase nova
    tokens = limpar_texto(frase)  # Aplica a limpeza e tokenização
    return vetor_frase(tokens)  # Converte a frase em vetor médio usando o modelo Word2Vec

vetores_novos = [vetorizacao_nova(f) for f in novas_frases]  # Aplica a vetorização para todas as frases novas
pred_novos = model.predict(vetores_novos)  # Realiza a predição dos noticias com o modelo treinado

for frase, pred in zip(novas_frases, pred_novos):  # Mostra cada frase com a respectivo noticia previsto
    print(f"Frase: {frase}\n→ Noticia prevista: {pred}\n")

Frase: STF define que jogar League of Legends é crime
→ Noticia prevista: Falsa

Frase: Primeira turma dos ministros do STF tem juntos 5018 anos de idade
→ Noticia prevista: Verdadeira

Frase: O STF obriga presidente a leitura
→ Noticia prevista: Verdadeira

Frase: Primeira turma do STF determina réus os investigados de 8 de janeiro
→ Noticia prevista: Falsa

Frase: STF aceita denuncia contra Bolsonaro
→ Noticia prevista: Falsa



## Balanceamento de classes com SMOTE

In [ ]:
# Codificar os rótulos para SMOTE
le = LabelEncoder()  # Cria um codificador para transformar os rótulos de texto em valores numéricos
y_encoded = le.fit_transform(y)  # Aplica a codificação (ex: 'Feliz' → 0, 'Triste' → 1)

# Aplicar SMOTE (k=1 pois a base é pequena)
smote = SMOTE(random_state=42, k_neighbors=1)  # Instancia o SMOTE com 1 vizinho, adequado para conjuntos pequenos
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)  # Aplica o SMOTE gerando exemplos sintéticos para balancear as classes

# Visualizar nova distribuição
Counter(le.inverse_transform(y_resampled))  # Mostra a contagem de exemplos por classe após o balanceamento (com os rótulos originais)

Counter({'Falsa': 50, 'Verdadeira': 50})

## Novo treinamento e avaliação com dados balanceados

In [ ]:
from imblearn.over_sampling import SMOTE  # ou RandomOverSampler, conforme sua escolha
smote = SMOTE(random_state=42)
le = LabelEncoder()
y_enc = le.fit_transform(y)
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)  # Divide os dados balanceados em treino (70%) e teste (30%)
model = GaussianNB()  # Cria uma instância do classificador Naive Bayes Gaussiano
model.fit(X_train, y_train)  # Treina o modelo com os dados de treino balanceados
y_pred = model.predict(X_test)  # Realiza a predição com o modelo treinado sobre os dados de teste

print("Acurácia:", accuracy_score(y_test, y_pred))  # Exibe a acurácia do modelo com os dados balanceados
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))  # Exibe a matriz de confusão com os dados balanceados
print("Relatório de Classificação:\n", classification_report(y_test, y_pred, target_names=le.classes_))  # Mostra o relatório com métricas para cada classe usando os nomes originais

Acurácia: 0.9
Matriz de Confusão:
 [[17  0]
 [ 3 10]]
Relatório de Classificação:
               precision    recall  f1-score   support

       Falsa       0.85      1.00      0.92        17
  Verdadeira       1.00      0.77      0.87        13

    accuracy                           0.90        30
   macro avg       0.93      0.88      0.89        30
weighted avg       0.91      0.90      0.90        30



## Predição com frases novas (com modelo balanceado)

In [ ]:
with open("teste.txt", "r", encoding="utf-8") as f:  # Abre o arquivo contendo frases para teste
    novas_frases = [linha.split(',')[0].strip("' ") for linha in f.readlines()]  # Lê cada linha, remove aspas e espaços extras

vetores_novos = [vetorizacao_nova(f) for f in novas_frases]  # Aplica a vetorização (Word2Vec) para cada nova frase
pred_novos = model.predict(vetores_novos)  # Usa o modelo treinado para prever o noticias das novas frases

for frase, pred in zip(novas_frases, pred_novos): # Itera sobre cada frase e seu noticias previsto
    print(f"Frase: {frase}\n→ notícia prevista: {pred}\n")  # Exibe a frase original e o rótulo previsto (convertido de volta para texto)

Frase: STF define que jogar League of Legends é crime
→ notícia prevista: Falsa

Frase: Primeira turma dos ministros do STF tem juntos 5018 anos de idade
→ notícia prevista: Falsa

Frase: O STF obriga presidente a leitura
→ notícia prevista: Falsa

Frase: Primeira turma do STF determina réus os investigados de 8 de janeiro
→ notícia prevista: Falsa

Frase: STF aceita denuncia contra Bolsonaro
→ notícia prevista: Falsa



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
